In [ ]:
!git clone https://github.com/anonymoususer007x2/AROMA

In [ ]:
!pip install openai
!pip install anthropic

In [ ]:
import openai
import anthropic
import pandas as pd
import re
import requests
import numpy as np
import os
from threading import Thread
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [ ]:
# file_path = '/content/AROMA/humanfeedback_category_proposer.csv'
# file_path='/content/AROMA/codex_ds1000_results'
# file_path='/content/AROMA/Claude_12_gsm8k_NO_cot.csv'
data=pd.read_csv(file_path)

Uncomment to retrieve data for the DS-1000 Pandas or Numpy results

In [ ]:
# data = data[data['library'] == 'Pandas'].copy()
# data = data[data['library'] == 'Numpy'].copy()

In [ ]:
incorrect_samples=data

Insert your key here

In [ ]:
openai.api_key = ''
client = anthropic.Anthropic(
    api_key="",
)

Filter to get only rows where 'correct' column is False. For the human feedback data uncomment the line with 'Llama_lost' to get samples LLaMA-3 70B was preferred or not to other models. You can also generate categories with all rows.


In [ ]:
incorrect_samples = incorrect_samples[incorrect_samples['is_correct'] == False]
# incorrect_samples = incorrect_samples[incorrect_samples['Llama_lost'] == True]
random_seed = 20
shuffled_incorrect_samples = incorrect_samples.sample(frac=1, random_state=random_seed)

**Category Proposer**

In [ ]:
def get_general_failures(incorrect_samples, chunk_size=60, num_chunks=None, process_all=False):
    total_items = len(incorrect_samples)
    all_responses = {}

    if process_all:
        num_chunks = total_items // chunk_size + (1 if total_items % chunk_size else 0)

    end_limit = min(num_chunks * chunk_size, total_items) if num_chunks else total_items
    max_attempts = 5

    for start_index in range(0, end_limit, chunk_size):
        for attempt in range(max_attempts):
            try:
                end_index = min(start_index + chunk_size, total_items)
                question_samples = incorrect_samples['question'][start_index:end_index].tolist()

                formatted_samples = '\n'.join([f"- {question}" for question in question_samples])

                prompt_text = """
                I will provide a series of data for you to remember. Subsequently, I will ask you some
                questions to test your performance! Here are questions for you to memorize.

                {}

                The above questions were incorrectly answered by the machine learning model.
                Using these specific examples, are there any tasks that are shared among
                the questions the model answers incorrectly?

                For each identified task, please offer a detailed and precise description.
                Highlight the particular characteristics of the tasks that the model failed to answer correctly.
                Do not describe why the model failed but rather the patterns within the questions.

                Please focus on achieving a high level of specificity and clarity in your categorization,
                steering clear of broad or vague descriptions such as:
                'Subject-Specific Knowledge Requirement', 'Task Ambiguity',
                'Context-Dependent Jargon Interpretation', 'Specialized Domain Knowledge Required', 'Complex Question Mishandling',
                'Ambiguously Phrased Question', 'Implicit Contextual Understanding'
                'Implicit Language Competency Assumption', 'Specific Language Processing Tasks', or
                'Inference-Based Questions.'


                We will evaluate the effectiveness and generalizability of the patterns descriptions using a separate set of unseen questions.
                Each description should be clear, precise, and comprehensive enough to allow us to reliably identify and categorize new questions
                that exhibit the same pattern characteristics, even if they differ in their specific content or phrasing.

                For each failure, please focus on isolating a single task among the questions,
                rather than grouping multiple tasks under a single failure mode.

                Instead of discussing the content of the questions within each failure mode description,
                please provide two short snippets of questions that demonstrate the identified task.

                The desired format for presenting each failure mode analysis is as follows:

                1. **Task Title**: Provide a detailed description of the specific pattern observed in the questions that were answered incorrectly.

                - Example Question 1 (Demonstrating Failure Mode 1)
                - Example Question 2 (Demonstrating Failure Mode 1)

                2. **Task Title**: Provide a detailed description of the specific pattern observed in the questions that were answered incorrectly.

                - Example Question 1 (Demonstrating Failure Mode 2)
                - Example Question 2 (Demonstrating Failure Mode 2)


                """.format(formatted_samples)

                response = openai.chat.completions.create(
                    model="gpt-4-1106-preview",
                    messages=[
                        {"role": "user", "content": prompt_text}
                    ],
                    temperature=1,
                    max_tokens=4000,
                    top_p=1,
                    frequency_penalty=0,
                    presence_penalty=0
                )
                # response = client.messages.create(
                #     model="claude-3-opus-20240229",
                #     messages=[
                #         {"role": "user", "content": prompt_text}
                #     ],
                #     temperature=1,
                #     max_tokens=3400,
                #     top_p=1,
                # )


                #openAI
                all_responses[f'chunk_{start_index}_{end_index}'] = response.choices[0].message.content

                #claude
                # all_responses[f'chunk_{start_index}_{end_index}'] = response.content[0].text.strip()

                break

            except openai.BadRequestError as e:
                if 'maximum context length' in str(e):
                    chunk_size = chunk_size - 5
                    if chunk_size <= 0:
                        raise ValueError("Chunk size reduced to zero, cannot proceed with requests.")
                else:
                    raise

            if (start_index // chunk_size + 1) >= num_chunks:
                break

    return all_responses


In [ ]:
response_dict = get_general_failures(shuffled_incorrect_samples, num_chunks=1)
for chunk_key, response_content in response_dict.items():
    print(f"Chunk: {chunk_key}")
    print("Response Content:")
    print(response_content)
    print("\n-----------------------\n")


In [ ]:
failures = pd.DataFrame(columns=["Failure Mode Title", "Failure Mode Description"])

In [ ]:
def process_chunk(chunk):
    data = []
    pattern = r"\d+\.\s\*\*(.*?)\*\*:\s(.*?)(?=\n\s+-|\Z)"

    matches = re.findall(pattern, chunk, re.DOTALL)

    for title, description in matches:
        title = title.strip()
        description = description.strip()
        description = re.sub(r"\n\s+-.*", "", description, flags=re.DOTALL).strip()
        data.append({"Failure Mode Title": title, "Failure Mode Description": description})

    df_chunk = pd.DataFrame(data)
    return df_chunk


In [ ]:
df_list = [process_chunk(chunk) for chunk_key, chunk in response_dict.items()]

failures_df = pd.concat(df_list, ignore_index=True)

In [ ]:
failures_df['Failure Mode Title'] = failures_df['Failure Mode Title'].str.replace('Failure Mode:', '', regex=False)


**The Falsifier**

In [ ]:
class ClaudeQueryProcessor:
    def __init__(self, client):
        """
        Initialize the ClaudeQueryProcessor instance with a Claude client.

        Parameters:
        - client: Initialized Claude Chat API client.
        """
        self.client = client

    def _interpret_response(self, response):
        """
        Interpret the Claude response to determine the alignment status of the question.
        """
        if 'Conclusion: Aligned' in response:
            return 'Yes'
        elif 'Conclusion: Unaligned' in response:
            return 'No'
        elif 'Conclusion: Uncertain' in response:
            return 'Uncertain'
        else:
            print(f"Unexpected response: {response}")
            return 'Error'

    def _save_responses(self, responses, file_path):
        """
        Save the responses DataFrame to a CSV file.
        """
        folder_path = os.path.dirname(file_path)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        responses.to_csv(file_path, index=False)
        print(f'Responses saved successfully to: {file_path}')

    def sample_questions_and_ask_claude(self, data, hypotheses, file_path):
        """
        Sample questions, ask Claude for evaluation, and save the responses.
        """
        responses = []

        for index, hypothesis_row in hypotheses.iterrows():
            failure_mode = hypothesis_row['Failure Mode Title']
            description = hypothesis_row['Failure Mode Description']
            print('failure mode:', failure_mode)

            for question in data['question'].sample(2):
                prompt_text = self._generate_prompt(failure_mode, description, question)
                response = self.ask_claude(prompt_text)
                print('model response',response)
                is_falsifiable = self._interpret_response(response)
                print('Response:', is_falsifiable)

                responses.append({
                    'Hypothesis': failure_mode,
                    'Hypoth Descript': description,
                    'Question': question,
                    'Response': response,
                    'Alignment Status': is_falsifiable
                })

        self._save_responses(pd.DataFrame(responses), file_path)
        return pd.DataFrame(responses)

    def _generate_prompt(self, failure_mode, description, question):
        """
        Generate the prompt text for Claude.
        """
        return f"""
        Before answering, provide your reasoning.

        Evaluate the alignment between the provided question and the hypothesis based on the title and description.
        Importantly, a question should be considered aligned even if the aspect connecting it to the criteria is not
        the primary focus of the question.

        The question isn't required to completely satisfy all the criteria. If any part of the question, no matter how straightforward or simple,
        meets any aspect of the criteria, it should be considered aligned.

        Additionally, assess if the hypothesis description is too vague, ambiguous, or broadly interpreted to make
        a confident judgment on alignment and explicitly state the reasons for this uncertainty.

        Hypothesized Failure Mode: "{failure_mode}"
        Description of Failure Mode: "{description}"
        Question: "{question}"

        1. Analysis: Examine the failure mode and its description along with the provided question. Identify any elements in the question that relate to the hypothesis, regardless of their prominence.

        2. Reasoning: Discuss the connections or discrepancies found during the analysis. Explain why these points lead you to consider the question as aligned, unaligned, or too ambiguous to determine.

        3. Conclusion: Provide your final assessment based on the analysis, reasoning, and hypothesis feedback.
        - If the question aligns with the hypothesis, state 'Conclusion: Aligned'.
        - If the question does not align with the hypothesis, state 'Conclusion: Unaligned'.
        - If it's unclear whether the question aligns due to vagueness or ambiguity in the hypothesis, state 'Conclusion: Uncertain'. If the hypothesis is deemed too vague or ambiguous, provide specific reasons and suggest how it might be refined.
        """

    def ask_claude(self, prompt_text):
        """
        Send a prompt to the Claude Chat API and return the response.
        """
        try:
            response = self.client.messages.create(
                model="claude-3-opus-20240229",
                messages=[
                    {"role": "user", "content": prompt_text}
                ],
                temperature=1,
                max_tokens=1000,
                top_p=1,
            )
            if response.content and len(response.content) > 0:
                content_text = response.content[0].text.strip()
                return content_text
            else:
                print("No content found in response.")
                return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

In [ ]:
file_path_1 = 'critiqued_hypoth.csv'

Add more threads for faster processing

In [ ]:
cq_processor1 = ClaudeQueryProcessor(client)

In [ ]:
df1=cq_processor1.sample_questions_and_ask_claude(data, failures_df[:1], file_path)

In [ ]:
sampled_responses=df1

In [ ]:
filtered_data = []

for hypothesis in sampled_responses['Hypothesis'].unique():
    temp_df = sampled_responses[sampled_responses['Hypothesis'] == hypothesis]
    uncertain_count = (temp_df['Alignment Status'] == 'Uncertain').sum()
    if uncertain_count <= 2:
        filtered_data.append(temp_df)

filtered_df = pd.concat(filtered_data, ignore_index=True)


In [ ]:
unique_hypotheses_2 = filtered_df['Hypothesis'].unique()

for hypothesis in unique_hypotheses_2:
    print(f"Hypothesis: {hypothesis}")
    filtered_df_2 = filtered_df[filtered_df['Hypothesis'] == hypothesis]
    alignment_status_counts = filtered_df_2['Alignment Status'].value_counts()
    print(alignment_status_counts)
    print("\n")

In [ ]:
copied_df = filtered_df[['Hypothesis', 'Hypoth Descript']].copy().rename(
    columns={'Hypothesis': 'Failure Mode Title', 'Hypoth Descript': 'Failure Mode Description'}
)


In [ ]:
copied_df_no_duplicates = copied_df.drop_duplicates()
critiqued_failures = copied_df_no_duplicates.reset_index(drop=True)

**Deduplication**

In [ ]:
api_url = ""

In [ ]:
def clean_description(description):
    """
    Removes the leading numbering and any special formatting from the description.

    :param description: A string representing the description.
    :return: A cleaned string of the description.
    """
    cleaned = re.sub(r'^\d+\.\s*\**', '', description).strip()
    return cleaned

def embed_descriptions(df, column_name, api_key, api_url):
    """
    Embeds each description in the dataframe column using OpenAI API.

    :param df: The dataframe containing the descriptions.
    :param column_name: The name of the column with descriptions to be embedded.
    :param api_key: OpenAI API key.
    :param api_url: OpenAI API URL for embeddings.
    :return: The dataframe with added column for embeddings.
    """
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    embeddings = []

    for description in df[column_name]:
        cleaned_description = clean_description(description)

        data = {
            "model": "text-embedding-ada-002",
            "input": cleaned_description
        }

        response = requests.post(api_url, headers=headers, json=data)

        if response.status_code == 200:
            embedding = response.json()['data']
            actual_embedding = embedding[0]['embedding']

            embeddings.append(actual_embedding)
        else:
            print(f"Error embedding description: {response.status_code} - {response.text}")
            embeddings.append(None)

    df['embedding'] = embeddings
    return df

In [ ]:
df = embed_descriptions(critiqued_failures, 'Failure Mode Title', openai.api_key, api_url)


In [ ]:
def remove_similar_rows(df, threshold=0.9):
    """
    Removes rows from the dataframe with cosine similarity higher than the given threshold.

    :param df: Dataframe with a column 'embedding' containing the embeddings.
    :param threshold: The cosine similarity threshold to remove similar rows.
    :return: A dataframe with highly similar rows removed.
    """
    embeddings_matrix = np.vstack(df['embedding'])
    similarity_matrix = cosine_similarity(embeddings_matrix)

    rows_to_drop = set()
    high_similarity_pairs = []

    for i in range(len(similarity_matrix)):
        for j in range(i+1, len(similarity_matrix)):
            if similarity_matrix[i, j] > threshold:
                rows_to_drop.add(j)
                high_similarity_pairs.append((df.iloc[i]['Failure Mode Title'], df.iloc[j]['Failure Mode Title'], similarity_matrix[i, j]))

    print("Failure Modes for rows to be dropped:\n")
    for index in rows_to_drop:
        print('Index:', index)
        print(df.iloc[index]['Failure Mode Title'])

    print("\nPairs of questions with high similarity scores:")
    for pair in high_similarity_pairs:
        print(f"Q1: {pair[0]}\nQ2: {pair[1]}\nSimilarity Score: {pair[2]}\n")

    # Drop identified rows
    df_filtered = df.drop(index=list(rows_to_drop))
    return df_filtered

In [ ]:
df_deduplicated = remove_similar_rows(df)


**The Validator**

In [ ]:
class HypothesisVerifier:
    def __init__(self, hypotheses, save_dir):
        self.hypotheses = hypotheses
        self.save_dir = save_dir
        self._prepare_save_dir()

    def ask_claude(self, prompt_text):
        """Send a prompt to the Claude Chat API and return the response."""
        try:
            response = client.messages.create(
                model="claude-3-opus-20240229",
                messages=[
                    {"role": "user", "content": prompt_text}
                ],
                temperature=1,
                max_tokens=1000,
                top_p=1,

            )
            if response.content and len(response.content) > 0:
                content_text = response.content[0].text.strip()
                return content_text
            else:
                print("No content found in response.")
                return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

    def _prepare_save_dir(self):
        if not os.path.exists(self.save_dir):
            os.makedirs(self.save_dir)


    def verify(self, hypothesis_index, matched_data, start_index):
        hypothesis, description = self._get_hypothesis_info(hypothesis_index)
        verifications = ['Pending'] * len(matched_data.loc[start_index:])
        full_responses = [''] * len(matched_data.loc[start_index:])
        processed_counter = start_index

        for position, (index, row) in enumerate(matched_data.loc[start_index:].iterrows()):
            processed_counter += 1
            response = self._query_openai(self._construct_prompt(row['question'], hypothesis, description))
            print('response', response)
            if response:
                verifications[position] = self._interpret_response(response)
                full_responses[position] = response
            else:
                print(f"API request failed for question {index}. Moving to the next question.")
                verifications[position] = 'Error'
                full_responses[position] = 'API request failed'

            self._handle_verification_update(matched_data, verifications, full_responses, start_index, processed_counter, hypothesis_index)

        return matched_data

    def _get_hypothesis_info(self, hypothesis_index):
        hypothesis_row = self.hypotheses.iloc[hypothesis_index]
        return hypothesis_row['Failure Mode Title'], hypothesis_row['Failure Mode Description']

    def _query_openai(self, prompt_text):
        try:
            response = self.ask_claude(prompt_text)
            return response
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

    def _construct_prompt(self, question_content, hypothesis, description):
        prompt = (
        f"Before answering, provide your reasoning."
        f"Your job is to determine if the question is asking for information described in the task.\n\n"
        f"Task Mode Title: \"{hypothesis}\"\n"
        f"Description of Task: \"{description}\"\n"
        f"Question: \"{question_content}\"\n\n"
        f"Instructions:\n"
        f"1. Determine if the question is asking for information or assistance related to the task.\n"
        f"2. Carefully read the question and identify any elements or phrases in the question that are relevant to"
        f"the task and its description.\n\n"
        f"3. Conclusion: State 'Conclusion: Yes' if your reasoning identifies a connection, "
        f"between the characteristics described in the task and the characteristics of the question."
        f"If, on the other hand, your analysis reveals no such relation, then state 'Conclusion: No'."
        )
        return prompt

    def _interpret_response(self, response):
        if 'Conclusion: Yes' in response:
            return 'Yes'
        elif 'Conclusion: No' in response:
            return 'No'
        else:
            print(f"Unexpected response: {response}")
            return 'Error'


    def _handle_verification_update(self, matched_data, verifications, full_responses, start_index, processed_counter, hypothesis_index):
        expected_length = len(matched_data.loc[start_index:])
        if len(verifications) != expected_length:
            self._adjust_verifications_length(verifications, expected_length)
            self._adjust_verifications_length(full_responses, expected_length)

        matched_data.loc[start_index:, 'verification'] = verifications
        matched_data.loc[start_index:, 'Full Response'] = full_responses
        if processed_counter % 100 == 0 or processed_counter == expected_length:
            self._save_results_to_file(matched_data, hypothesis_index, processed_counter)

    def _adjust_verifications_length(self, verifications, expected_length):
        if len(verifications) > expected_length:
            return verifications[:expected_length]
        else:
            verifications.extend(['Error'] * (expected_length - len(verifications)))

    def _save_results_to_file(self, matched_data, hypothesis_index, processed_counter):
        filename = f'hypothesis_results_{hypothesis_index}_{processed_counter}.csv'
        matched_data.to_csv(os.path.join(self.save_dir, filename))
        print(f'Results saved to {filename}')

In [ ]:
retrieval_database=pd.read_csv('/content/AROMA/human_feedback_category_scorer_data.csv')

In [ ]:
verifier1 = HypothesisVerifier(df_deduplicated, save_dir='/content/')

final_df=verifier1.verify(hypothesis_index=0, matched_data=retrieval_database, start_index=1317)

In [ ]:
value_counts = final_df['is_correct'].value_counts()
true_count = value_counts.get(True, 0)
false_count = value_counts.get(False, 0)

total_count = true_count + false_count
score = true_count / total_count if total_count > 0 else 0

print(f"Score: {score:.2f}")
print(f"True count: {true_count}")
print(f"False count: {false_count}")

In [ ]:
verified_data=final_df[final_df['verification']=='Yes'].copy()

verified_data_value_counts = verified_data['is_correct'].value_counts()
verified_data_true_count = verified_data_value_counts.get(True, 0)
verified_data_false_count = verified_data_value_counts.get(False, 0)

verified_data_total_count = verified_data_true_count + verified_data_false_count
verified_data_score = verified_data_true_count / verified_data_total_count if verified_data_total_count > 0 else 0

print(f"Score: {verified_data_score:.2f}")
print(f"True count: {verified_data_true_count}")
print(f"False count: {verified_data_false_count}")